In [10]:
import numpy as np
import matplotlib.pyplot as plt

from algorithms.gam import gam
from algorithms.lv_hba import lv_hba
from algorithms.blocc import blocc

from utils import *

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [11]:
plt.rcParams.update({'font.size': 32})
plt.rcParams['font.sans-serif']=['Times New Roman']
plt.rcParams['axes.unicode_minus']=False
cmap = plt.get_cmap("tab10")

In [12]:
epochs = 80
seeds = 10

In [13]:
hparams = {
    'lv_hba': {
        'alpha': 0.01,
        'gama1': 0.1,
        'gama2': 0.1,
        'yita': 0.001
    },
    'gam': {
        'alpha': 0.05,
        'epsilon': 0.005
    },
    'blocc': {
        'gam': 10,
        'eta': 0.01
    }
}

# Dataset Diabetes

In [14]:
dataset_name = "diabetes"
data = load_diabetes()

n_train = 500
n_val = 150

In [16]:
x_train, y_train, x_val, y_val, x_test, y_test = train_val_test_split(data, 10, n_train, n_val)
x_train.shape, x_val.shape, x_test.shape

((500, 45), (150, 45), (118, 45))

In [ ]:
metrics_diabetes = {}
variables_diabetes = {}

algs = ['blocc', 'lv_hba', 'gam']

for a in algs:
    metrics_diabetes[a] = []
    variables_diabetes[a] = []

for seed in range(seeds):

    print(f"Starting seed {seed+1}/{seeds}: ", end="")

    x_train, y_train, x_val, y_val, x_test, y_test = train_val_test_split(data, seed, n_train, n_val)

    for alg in algs:
        print(f"{alg}", end=" ")

        metrics_seed, variables_seed = eval(alg)(x_train, y_train, x_val, y_val, x_test, y_test, hparams[alg], epochs, verbose=False)
        metrics_diabetes[alg].append(metrics_seed)
        variables_diabetes[alg].append(variables_seed)
    print("DONE")

## Evaluation

In [ ]:
results_path = 'Plots_diabetes/'# f'../results/hyperparam_opt/{dataset_name}/'

In [ ]:
metrics_lvhba = metrics_diabetes['lv_hba']
metrics_gam = metrics_diabetes['gam']
metrics_blocc = metrics_diabetes['blocc']

axis_lvhba_all = np.array([[x['time_computation'] for x in metrics] for metrics in metrics_lvhba])
axis_gam_all = np.array([[x['time_computation'] for x in metrics] for metrics in metrics_gam])
axis_blocc_all = np.array([[x['time_computation'] for x in metrics] for metrics in metrics_blocc])

axis_lvhba = axis_lvhba_all.mean(0)
axis_gam = axis_gam_all.mean(0)
axis_blocc = axis_blocc_all.mean(0)

axis_iter = np.arange(epochs)

In [ ]:
train_acc_lvhba = np.array([[x['train_acc'] for x in metric] for metric in metrics_lvhba])
train_acc_ga = np.array([[x['train_acc'] for x in metric] for metric in metrics_gam])
train_acc_blocc = np.array([[x['train_acc_F'] for x in metric] for metric in metrics_blocc])

val_acc_lvhba = np.array([[x['val_acc'] for x in metric] for metric in metrics_lvhba])
val_acc_ga = np.array([[x['val_acc'] for x in metric] for metric in metrics_gam])
val_acc_blocc = np.array([[x['val_acc_F'] for x in metric] for metric in metrics_blocc])

test_acc_lvhba = np.array([[x['test_acc'] for x in metric] for metric in metrics_lvhba])
test_acc_ga = np.array([[x['test_acc'] for x in metric] for metric in metrics_gam])
test_acc_blocc = np.array([[x['test_acc_F'] for x in metric] for metric in metrics_blocc])

In [ ]:
val_loss_lvhba = np.array([[x['val_loss'] for x in metric] for metric in metrics_lvhba])[:,0:]
val_loss_ga = np.array([[x['val_loss'] for x in metric] for metric in metrics_gam])[:,0:]
val_loss_blocc = np.array([[x['val_loss_F'] for x in metric] for metric in metrics_blocc])[:,0:]

test_loss_lvhba = np.array([[x['test_loss'] for x in metric] for metric in metrics_lvhba])[:,0:]
test_loss_ga = np.array([[x['test_loss'] for x in metric] for metric in metrics_gam])[:,0:]
test_loss_blocc = np.array([[x['test_loss_F'] for x in metric] for metric in metrics_blocc])[:,0:]

lower_loss_lvhba = np.array([[x['loss_lower'] for x in metric] for metric in metrics_lvhba])[:,1:]
lower_loss_ga = np.array([[x['loss_lower'] for x in metric] for metric in metrics_gam])[:,1:]
lower_loss_blocc = np.array([[x['loss_lower'] for x in metric] for metric in metrics_blocc])[:,1:]

# constraint_violation_lvhba = np.array([[x['constraint_violation'] for x in metric] for metric in metrics_lvhba])[:,1:]
# constraint_violation_blocc = np.array([[x['constraint_violation'] for x in metric] for metric in metrics_blocc])[:,1:]

### Measuring Convergence Time
When `(UpperLoss_{t+1}-UpperLoss_t)^2` is smaller than `eps = 1e-5`. We say convergence happen.

In [ ]:
print("-------------- Accuracy -------------")
print('blocc: {:0.3f} ({:0.3f})'.format(test_acc_blocc[:,-1].mean(),test_acc_blocc[:,-1].std()))
print('lvhba: {:0.3f} ({:0.3f})'.format(test_acc_lvhba[:,-1].mean(),test_acc_lvhba[:,-1].std()))
print('gam: {:0.3f} ({:0.3f})'.format(test_acc_ga[:,-1].mean(),test_acc_ga[:,-1].std()))

In [ ]:
eps = 1e-5

diff_val_loss_blocc = np.diff(val_loss_blocc,axis=1)**2
diff_val_loss_blocc[diff_val_loss_blocc<eps]=0
converged_id_blocc = np.argmin(diff_val_loss_blocc,axis = 1)

diff_val_loss_lvhba = np.diff(val_loss_lvhba,axis=1)**2
diff_val_loss_lvhba[diff_val_loss_lvhba<eps]=0
converged_id_lvhba = np.argmin(diff_val_loss_lvhba,axis = 1)

diff_val_loss_ga = np.diff(val_loss_ga,axis=1)**2
diff_val_loss_ga[diff_val_loss_ga<eps]=0
converged_id_ga = np.argmin(diff_val_loss_ga,axis = 1)

time_blocc = []
time_lvhba = []
time_ga = []
for i in range(seeds):
    time_blocc.append(axis_blocc_all[i,converged_id_blocc[i]+1])
    time_lvhba.append(axis_lvhba_all[i,converged_id_lvhba[i]+1])
    time_ga.append(axis_gam_all[i,converged_id_ga[i]+1])

time_blocc = np.array(time_blocc)
time_lvhba = np.array(time_lvhba)
time_ga = np.array(time_ga)
print("-------------- Running Time -------------")
print('blocc: {:0.3f} ({:0.3f})'.format(time_blocc.mean(),time_blocc.std()))
print('lvhba: {:0.3f} ({:0.3f})'.format(time_lvhba.mean(),time_lvhba.std()))
print('gam: {:0.3f} ({:0.3f})'.format(time_ga.mean(),time_ga.std()))


### Plots

In [ ]:
plt.figure(figsize=(8,6))

plt.plot(axis_lvhba,val_acc_lvhba.mean(0),'-', color = cmap(0),linewidth = 4, label="LV-HBA")
plt.fill_between(axis_lvhba,val_acc_lvhba.mean(0)-val_acc_lvhba.std(0),val_acc_lvhba.mean(0)+val_acc_lvhba.std(0), color = cmap(0),alpha=0.2)

plt.plot(axis_gam,val_acc_ga.mean(0),'--', color = cmap(2), linewidth = 4, label="GAM")
plt.fill_between(axis_gam,val_acc_ga.mean(0)-val_acc_ga.std(0),val_acc_ga.mean(0)+val_acc_ga.std(0), color = cmap(2), alpha=0.2)

plt.plot(axis_blocc,val_acc_blocc.mean(0),'-',color='r',linewidth = 4, label="BLOCC")
plt.fill_between(axis_blocc,val_acc_blocc.mean(0)-val_acc_blocc.std(0),val_acc_blocc.mean(0)+val_acc_blocc.std(0),color='r', alpha=0.2)

# Get the handles and labels
handles, labels = plt.gca().get_legend_handles_labels()

# Reorder the handles and labels as needed
order = [2,0,1]
ordered_handles = [handles[i] for i in order]
ordered_labels = [labels[i] for i in order]

# Create the legend with the new order
plt.legend(ordered_handles, ordered_labels)
              
plt.title('Validation Accuracy')
plt.xlabel('Running time /s')
plt.ylabel("Accuracy")
plt.xlim([0,axis_lvhba.max()])
plt.savefig(results_path + 'val_accuracy_running_time.pdf', bbox_inches="tight")

In [ ]:
plt.figure(figsize=(8,6))

iter_axis = np.arange(val_acc_ga.shape[1])


plt.plot(axis_iter,val_acc_lvhba.mean(0),'-',color = cmap(0), linewidth = 4,  label="LV-HBA")
plt.fill_between(axis_iter,val_acc_lvhba.mean(0)-val_acc_lvhba.std(0),val_acc_lvhba.mean(0)+val_acc_lvhba.std(0),color = cmap(0), alpha=0.2)

plt.plot(axis_iter,val_acc_ga.mean(0),'--',color = cmap(2),  linewidth = 4, label="GAM")
plt.fill_between(axis_iter,val_acc_ga.mean(0)-val_acc_ga.std(0),val_acc_ga.mean(0)+val_acc_ga.std(0),color = cmap(2), alpha=0.2)

plt.plot(axis_iter,val_acc_blocc.mean(0),'-',color = 'r', linewidth = 4, label="BLOCC")
plt.fill_between(axis_iter,val_acc_blocc.mean(0)-val_acc_blocc.std(0),val_acc_blocc.mean(0)+val_acc_blocc.std(0),color = 'r',alpha=0.2)

plt.legend(ordered_handles, ordered_labels)

plt.title('Validation Accuracy')
plt.xlabel('Iteration Count')
plt.ylabel("Accuracy")


plt.savefig(results_path + 'val_accuracy_iteration.pdf', bbox_inches="tight")

In [ ]:
plt.figure(figsize=(8,6))

plt.plot(axis_lvhba,test_acc_lvhba.mean(0),'-',linewidth = 4, color = cmap(0),label="LV-HBA")
plt.fill_between(axis_lvhba,test_acc_lvhba.mean(0)-test_acc_lvhba.std(0),test_acc_lvhba.mean(0)+test_acc_lvhba.std(0),color = cmap(0),alpha=0.2)

plt.plot(axis_gam,test_acc_ga.mean(0),'--',linewidth = 4, color = cmap(2),label="GAM")
plt.fill_between(axis_gam,test_acc_ga.mean(0)-test_acc_ga.std(0),test_acc_ga.mean(0)+test_acc_ga.std(0),color = cmap(2),alpha=0.2)

plt.plot(axis_blocc,test_acc_blocc.mean(0),'-',linewidth = 4, color = 'r',label="BLOCC")
plt.fill_between(axis_blocc,test_acc_blocc.mean(0)-test_acc_blocc.std(0),test_acc_blocc.mean(0)+test_acc_blocc.std(0),color = 'r',alpha=0.2)

# Get the handles and labels
handles, labels = plt.gca().get_legend_handles_labels()
# Reorder the handles and labels as needed
order = [2,0,1]
ordered_handles = [handles[i] for i in order]
ordered_labels = [labels[i] for i in order]
# Create the legend with the new order
plt.legend(ordered_handles, ordered_labels)

plt.title('Test Accuracy')
plt.xlabel('Running time /s')
plt.ylabel("Accuracy")

plt.xlim([0,axis_lvhba.max()])
plt.savefig(results_path + 'test_accuracy_running_time.pdf', bbox_inches="tight")

In [ ]:
plt.figure(figsize=(8,6))

plt.plot(axis_iter,test_acc_lvhba.mean(0),'-',color = cmap(0),linewidth = 4, label="LV-HBA")
plt.fill_between(axis_iter,test_acc_lvhba.mean(0)-test_acc_lvhba.std(0),test_acc_lvhba.mean(0)+test_acc_lvhba.std(0),color = cmap(0),alpha=0.2)

plt.plot(axis_iter,test_acc_ga.mean(0),'--',color = cmap(2),linewidth = 4, label="GAM")
plt.fill_between(axis_iter,test_acc_ga.mean(0)-test_acc_ga.std(0),test_acc_ga.mean(0)+test_acc_ga.std(0),color = cmap(2),alpha=0.2)


plt.plot(axis_iter,test_acc_blocc.mean(0),'-',color = 'r',linewidth = 4, label="BLOCC")
plt.fill_between(axis_iter,test_acc_blocc.mean(0)-test_acc_blocc.std(0),test_acc_blocc.mean(0)+test_acc_blocc.std(0),color = 'r',alpha=0.2)


plt.legend(ordered_handles, ordered_labels)

plt.title('Test Accuracy')
plt.xlabel('Iteration Count')
plt.ylabel("Accuracy")
plt.savefig(results_path + 'test_accuracy_iteration.pdf', bbox_inches="tight")

In [ ]:
plt.figure(figsize=(8,6))


plt.plot(iter_axis,val_loss_lvhba.mean(0),'-',color = cmap(0), linewidth = 4, label="LV-HBA")
fill_dn_LV = val_loss_lvhba.mean(0)-val_loss_lvhba.std(0)
fill_up_LV = val_loss_lvhba.mean(0)+val_loss_lvhba.std(0)
plt.fill_between(iter_axis[1:],fill_dn_LV[1:],fill_up_LV[1:],color = cmap(0), alpha=0.2)

plt.plot(iter_axis,val_loss_ga.mean(0),'--',color = cmap(2), linewidth = 4, label="GAM")
fill_up_GAM = val_loss_ga.mean(0)-val_loss_ga.std(0)
fill_dn_GAM = val_loss_ga.mean(0)+val_loss_ga.std(0)
plt.fill_between(iter_axis[1:],fill_dn_GAM[1:],fill_up_GAM[1:],color = cmap(2), alpha=0.2)

plt.plot(iter_axis,val_loss_blocc.mean(0),'-',color = 'r', linewidth = 4, label="BLOCC")
fill_dn_blocc = val_loss_blocc.mean(0)-val_loss_blocc.std(0)
fill_up_blocc = val_loss_blocc.mean(0)+val_loss_blocc.std(0)
plt.fill_between(iter_axis[1:],fill_dn_blocc[1:],fill_up_blocc[1:],color = 'r', alpha=0.2)

plt.legend(ordered_handles, ordered_labels)

plt.title('Validation Loss (Upper Level)')
plt.xlabel('Iteration Count')
plt.ylabel("Loss")


plt.ylim([1.5,3.4])
plt.savefig(results_path + 'upper_obj_iteration.pdf', bbox_inches="tight")

In [ ]:
plt.figure(figsize=(8,6))


plt.plot(axis_lvhba,val_loss_lvhba.mean(0),'-',color = cmap(0), linewidth = 4, label="LV-HBA")
fill_dn_LV = val_loss_lvhba.mean(0)-val_loss_lvhba.std(0)
fill_up_LV = val_loss_lvhba.mean(0)+val_loss_lvhba.std(0)
plt.fill_between(axis_lvhba[1:],fill_dn_LV[1:],fill_up_LV[1:],color = cmap(0), alpha=0.2)

plt.plot(axis_gam,val_loss_ga.mean(0),'--',color = cmap(2), linewidth = 4, label="GAM")
fill_up_GAM = val_loss_ga.mean(0)-val_loss_ga.std(0)
fill_dn_GAM = val_loss_ga.mean(0)+val_loss_ga.std(0)
plt.fill_between(axis_gam[1:],fill_dn_GAM[1:],fill_up_GAM[1:],color = cmap(2), alpha=0.2)

plt.plot(axis_blocc,val_loss_blocc.mean(0),'-',color = 'r', linewidth = 4, label="BLOCC")
fill_dn_blocc = val_loss_blocc.mean(0)-val_loss_blocc.std(0)
fill_up_blocc = val_loss_blocc.mean(0)+val_loss_blocc.std(0)
plt.fill_between(axis_blocc[1:],fill_dn_blocc[1:],fill_up_blocc[1:],color = 'r', alpha=0.2)

plt.legend(ordered_handles, ordered_labels)
plt.title('Validation Loss (Upper Level)')
plt.xlabel('Running Time /s')
plt.ylabel("Loss")

plt.ylim([1.5,4])
plt.xlim([0,13])
plt.xlim([0,axis_lvhba.max()])
plt.savefig(results_path + 'upper_obj_time.pdf', bbox_inches="tight")

In [ ]:
plt.figure(figsize=(8,6))


plt.plot(iter_axis,test_loss_lvhba.mean(0),'-',color = cmap(0), linewidth = 4, label="LV-HBA")
fill_dn_LV = test_loss_lvhba.mean(0)-test_loss_lvhba.std(0)
fill_up_LV = test_loss_lvhba.mean(0)+test_loss_lvhba.std(0)
plt.fill_between(iter_axis[1:],fill_dn_LV[1:],fill_up_LV[1:],color = cmap(0), alpha=0.2)

plt.plot(iter_axis,test_loss_ga.mean(0),'--',color = cmap(2), linewidth = 4, label="GAM")
fill_up_GAM = test_loss_ga.mean(0)-test_loss_ga.std(0)
fill_dn_GAM = test_loss_ga.mean(0)+test_loss_ga.std(0)
plt.fill_between(iter_axis[1:],fill_dn_GAM[1:],fill_up_GAM[1:],color = cmap(2), alpha=0.2)

plt.plot(iter_axis,test_loss_blocc.mean(0),'-',color = 'r', linewidth = 4, label="BLOCC")
fill_dn_blocc = test_loss_blocc.mean(0)-test_loss_blocc.std(0)
fill_up_blocc = test_loss_blocc.mean(0)+test_loss_blocc.std(0)
plt.fill_between(iter_axis[1:],fill_dn_blocc[1:],fill_up_blocc[1:],color = 'r', alpha=0.2)
plt.legend()
plt.title('Test Loss')
plt.xlabel('Iteration Count')
plt.ylabel("Loss")

plt.ylim([1.5,3.4])
plt.savefig(results_path + 'test_obj_iteration.pdf', bbox_inches="tight")

In [ ]:
plt.figure(figsize=(8,6))

plt.plot(iter_axis[1:],lower_loss_blocc.mean(0),'-',color = 'r', linewidth = 4, label="BLOCC")
# fill_dn_blocc = lower_loss_blocc.mean(0)-lower_loss_blocc.std(0)
# fill_up_blocc = lower_loss_blocc.mean(0)+lower_loss_blocc.std(0)
# plt.fill_between(iter_axis[1:],fill_dn_blocc,fill_up_blocc,alpha=0.2)
# the above didn't work as the standard deviation is too small

plt.plot(iter_axis[1:],lower_loss_lvhba.mean(0),'-',color = cmap(0), linewidth = 4, label="LV-HBA")
fill_dn_LV = lower_loss_lvhba.mean(0)-lower_loss_lvhba.std(0)
fill_up_LV = lower_loss_lvhba.mean(0)+lower_loss_lvhba.std(0)
plt.fill_between(iter_axis[1:],fill_dn_LV,fill_up_LV,color = cmap(0),alpha=0.2)

plt.plot(iter_axis[1:],lower_loss_ga.mean(0),'--', color = cmap(2),linewidth = 4, label="GAM")
fill_up_GAM = lower_loss_ga.mean(0)-lower_loss_ga.std(0)
fill_dn_GAM = lower_loss_ga.mean(0)+lower_loss_ga.std(0)
plt.fill_between(iter_axis[1:],fill_dn_GAM,fill_up_GAM,color = cmap(2),alpha=0.2)

plt.legend()
plt.title('Lower Level Objective')
plt.xlabel('Iteration Count')
plt.ylabel("Loss")

plt.savefig(results_path + 'lower_obj_iteration.pdf', bbox_inches="tight")

In [ ]:
plt.figure(figsize=(8,6))

plt.plot(axis_blocc[1:],lower_loss_blocc.mean(0),'-',color = 'r', linewidth = 4, label="BLOCC")
# fill_dn_blocc = lower_loss_blocc.mean(0)-lower_loss_blocc.std(0)
# fill_up_blocc = lower_loss_blocc.mean(0)+lower_loss_blocc.std(0)
# plt.fill_between(iter_axis[1:],fill_dn_blocc,fill_up_blocc,alpha=0.2)
# the above didn't work as the standard deviation is too small

plt.plot(axis_lvhba[1:],lower_loss_lvhba.mean(0),'-',color = cmap(0), linewidth = 4, label="LV-HBA")
fill_dn_LV = lower_loss_lvhba.mean(0)-lower_loss_lvhba.std(0)
fill_up_LV = lower_loss_lvhba.mean(0)+lower_loss_lvhba.std(0)
plt.fill_between(axis_lvhba[1:],fill_dn_LV,fill_up_LV,color = cmap(0),alpha=0.2)

plt.plot(axis_gam[1:],lower_loss_ga.mean(0),'--', color = cmap(2),linewidth = 4, label="GAM")
fill_up_GAM = lower_loss_ga.mean(0)-lower_loss_ga.std(0)
fill_dn_GAM = lower_loss_ga.mean(0)+lower_loss_ga.std(0)
plt.fill_between(axis_gam[1:],fill_dn_GAM,fill_up_GAM,color = cmap(2),alpha=0.2)

plt.legend()
plt.title('Lower Level Objective')
plt.xlabel('Running Time /s')
plt.ylabel("Loss")
plt.xlim([0,axis_lvhba.max()])

plt.savefig(results_path + 'lower_obj_time.pdf', bbox_inches="tight")

# Dataset Fourclass

In [ ]:
dataset_name = "fourclass"
data = load_fourclass()

n_train = 500
n_val = 150

In [ ]:
metrics_fourclass = {}
variables_fourclass = {}

algs = ['blocc', 'lv_hba', 'gam']

for a in algs:
    metrics_fourclass[a] = []
    variables_fourclass[a] = []

for seed in range(seeds):

    print(f"Starting seed {seed+1}/{seeds}: ", end="")

    x_train, y_train, x_val, y_val, x_test, y_test = train_val_test_split(data, seed, n_train, n_val)

    for alg in algs:
        print(f"{alg}", end=" ")

        metrics_seed, variables_seed = eval(alg)(x_train, y_train, x_val, y_val, x_test, y_test, hparams[alg], epochs, verbose=False)
        metrics_fourclass[alg].append(metrics_seed)
        variables_fourclass[alg].append(variables_seed)
    print("DONE")

## Evaluation

In [ ]:
results_path = 'Plots_fourclass/'#f'../results/hyperparam_opt/{dataset_name}/'

In [ ]:
metrics_lvhba = metrics_fourclass['lv_hba']
metrics_gam = metrics_fourclass['gam']
metrics_blocc = metrics_fourclass['blocc']


axis_lvhba_all = np.array([[x['time_computation'] for x in metrics] for metrics in metrics_lvhba])
axis_gam_all = np.array([[x['time_computation'] for x in metrics] for metrics in metrics_gam])
axis_blocc_all = np.array([[x['time_computation'] for x in metrics] for metrics in metrics_blocc])

axis_lvhba = axis_lvhba_all.mean(0)
axis_gam = axis_gam_all.mean(0)
axis_blocc = axis_blocc_all.mean(0)

axis_iter = np.arange(epochs)

In [ ]:
train_acc_lvhba = np.array([[x['train_acc'] for x in metrics] for metrics in metrics_lvhba])
train_acc_ga = np.array([[x['train_acc'] for x in metrics] for metrics in metrics_gam])
train_acc_blocc = np.array([[x['train_acc_F'] for x in metrics] for metrics in metrics_blocc])

val_acc_lvhba = np.array([[x['val_acc'] for x in metrics] for metrics in metrics_lvhba])
val_acc_ga = np.array([[x['val_acc'] for x in metrics] for metrics in metrics_gam])
val_acc_blocc = np.array([[x['val_acc_F'] for x in metrics] for metrics in metrics_blocc])

test_acc_lvhba = np.array([[x['test_acc'] for x in metrics] for metrics in metrics_lvhba])
test_acc_ga = np.array([[x['test_acc'] for x in metrics] for metrics in metrics_gam])
test_acc_blocc = np.array([[x['test_acc_F'] for x in metrics] for metrics in metrics_blocc])

In [ ]:
val_loss_lvhba = np.array([[x['val_loss'] for x in metrics] for metrics in metrics_lvhba])[:,0:]
val_loss_ga = np.array([[x['val_loss'] for x in metrics] for metrics in metrics_gam])[:,0:]
val_loss_blocc = np.array([[x['val_loss_F'] for x in metrics] for metrics in metrics_blocc])[:,0:]

test_loss_lvhba = np.array([[x['test_loss'] for x in metrics] for metrics in metrics_lvhba])[:,0:]
test_loss_ga = np.array([[x['test_loss'] for x in metrics] for metrics in metrics_gam])[:,0:]
test_loss_blocc = np.array([[x['test_loss_F'] for x in metrics] for metrics in metrics_blocc])[:,0:]

lower_loss_lvhba = np.array([[x['loss_lower'] for x in metrics] for metrics in metrics_lvhba])[:,1:]
lower_loss_ga = np.array([[x['loss_lower'] for x in metrics] for metrics in metrics_gam])[:,1:]
lower_loss_blocc = np.array([[x['loss_lower'] for x in metrics] for metrics in metrics_blocc])[:,1:]


### Measuring Convergence Time
When `(UpperLoss_{t+1}-UpperLoss_t)^2` is smaller than `eps = 1e-10`. We say convergence happen.

In [ ]:
print("-------------- Accuracy -------------")
print('blocc: {:0.3f} ({:0.3f})'.format(test_acc_blocc[:,-1].mean(),test_acc_blocc[:,-1].std()))
print('lvhba: {:0.3f} ({:0.3f})'.format(test_acc_lvhba[:,-1].mean(),test_acc_lvhba[:,-1].std()))
print('gam: {:0.3f} ({:0.3f})'.format(test_acc_ga[:,-1].mean(),test_acc_ga[:,-1].std()))

In [ ]:
eps = 1e-5

diff_val_loss_blocc = np.diff(val_loss_blocc,axis=1)**2
diff_val_loss_blocc[diff_val_loss_blocc<eps]=0
converged_id_blocc = np.argmin(diff_val_loss_blocc,axis = 1)

diff_val_loss_lvhba = np.diff(val_loss_lvhba,axis=1)**2
diff_val_loss_lvhba[diff_val_loss_lvhba<eps]=0
converged_id_lvhba = np.argmin(diff_val_loss_lvhba,axis = 1)

diff_val_loss_ga = np.diff(val_loss_ga,axis=1)**2
diff_val_loss_ga[diff_val_loss_ga<eps]=0
converged_id_ga = np.argmin(diff_val_loss_ga,axis = 1)

time_blocc = []
time_lvhba = []
time_ga = []
for i in range(seeds):
    time_blocc.append(axis_blocc_all[i,converged_id_blocc[i]+1])
    time_lvhba.append(axis_lvhba_all[i,converged_id_lvhba[i]+1])
    time_ga.append(axis_gam_all[i,converged_id_ga[i]+1])

time_blocc = np.array(time_blocc)
time_lvhba = np.array(time_lvhba)
time_ga = np.array(time_ga)
print("-------------- Running Time -------------")
print('blocc: {:0.3f} ({:0.3f})'.format(time_blocc.mean(),time_blocc.std()))
print('lvhba: {:0.3f} ({:0.3f})'.format(time_lvhba.mean(),time_lvhba.std()))
print('gam: {:0.3f} ({:0.3f})'.format(time_ga.mean(),time_ga.std()))

### Plots

In [ ]:
plt.figure(figsize=(8,6))

plt.plot(axis_lvhba,val_acc_lvhba.mean(0),'-', color = cmap(0),linewidth = 4, label="LV-HBA")
plt.fill_between(axis_lvhba,val_acc_lvhba.mean(0)-val_acc_lvhba.std(0),val_acc_lvhba.mean(0)+val_acc_lvhba.std(0), color = cmap(0),alpha=0.2)

plt.plot(axis_gam,val_acc_ga.mean(0),'--', color = cmap(2), linewidth = 4, label="GAM")
plt.fill_between(axis_gam,val_acc_ga.mean(0)-val_acc_ga.std(0),val_acc_ga.mean(0)+val_acc_ga.std(0), color = cmap(2), alpha=0.2)

plt.plot(axis_blocc,val_acc_blocc.mean(0),'-',color='r',linewidth = 4, label="BLOCC")
plt.fill_between(axis_blocc,val_acc_blocc.mean(0)-val_acc_blocc.std(0),val_acc_blocc.mean(0)+val_acc_blocc.std(0),color='r', alpha=0.2)

# Get the handles and labels
handles, labels = plt.gca().get_legend_handles_labels()

# Reorder the handles and labels as needed
order = [2,0,1]
ordered_handles = [handles[i] for i in order]
ordered_labels = [labels[i] for i in order]

# Create the legend with the new order
plt.legend(ordered_handles, ordered_labels)
              
plt.title('Validation Accuracy')
plt.xlabel('Running time /s')
plt.ylabel("Accuracy")
plt.xlim([0,axis_lvhba.max()])
plt.savefig(results_path + 'val_accuracy_running_time.pdf', bbox_inches="tight")

In [ ]:
plt.figure(figsize=(8,6))

iter_axis = np.arange(val_acc_ga.shape[1])


plt.plot(axis_iter,val_acc_lvhba.mean(0),'-',color = cmap(0), linewidth = 4,  label="LV-HBA")
plt.fill_between(axis_iter,val_acc_lvhba.mean(0)-val_acc_lvhba.std(0),val_acc_lvhba.mean(0)+val_acc_lvhba.std(0),color = cmap(0), alpha=0.2)

plt.plot(axis_iter,val_acc_ga.mean(0),'--',color = cmap(2),  linewidth = 4, label="GAM")
plt.fill_between(axis_iter,val_acc_ga.mean(0)-val_acc_ga.std(0),val_acc_ga.mean(0)+val_acc_ga.std(0),color = cmap(2), alpha=0.2)

plt.plot(axis_iter,val_acc_blocc.mean(0),'-',color = 'r', linewidth = 4, label="BLOCC")
plt.fill_between(axis_iter,val_acc_blocc.mean(0)-val_acc_blocc.std(0),val_acc_blocc.mean(0)+val_acc_blocc.std(0),color = 'r',alpha=0.2)

plt.legend(ordered_handles, ordered_labels)

plt.title('Validation Accuracy')
plt.xlabel('Iteration Count')
plt.ylabel("Accuracy")


plt.savefig(results_path + 'val_accuracy_iteration.pdf', bbox_inches="tight")

In [ ]:
plt.figure(figsize=(8,6))

plt.plot(axis_lvhba,test_acc_lvhba.mean(0),'-',linewidth = 4, color = cmap(0),label="LV-HBA")
plt.fill_between(axis_lvhba,test_acc_lvhba.mean(0)-test_acc_lvhba.std(0),test_acc_lvhba.mean(0)+test_acc_lvhba.std(0),color = cmap(0),alpha=0.2)

plt.plot(axis_gam,test_acc_ga.mean(0),'--',linewidth = 4, color = cmap(2),label="GAM")
plt.fill_between(axis_gam,test_acc_ga.mean(0)-test_acc_ga.std(0),test_acc_ga.mean(0)+test_acc_ga.std(0),color = cmap(2),alpha=0.2)

plt.plot(axis_blocc,test_acc_blocc.mean(0),'-',linewidth = 4, color = 'r',label="BLOCC")
plt.fill_between(axis_blocc,test_acc_blocc.mean(0)-test_acc_blocc.std(0),test_acc_blocc.mean(0)+test_acc_blocc.std(0),color = 'r',alpha=0.2)

# Get the handles and labels
handles, labels = plt.gca().get_legend_handles_labels()
# Reorder the handles and labels as needed
order = [2,0,1]
ordered_handles = [handles[i] for i in order]
ordered_labels = [labels[i] for i in order]
# Create the legend with the new order
plt.legend(ordered_handles, ordered_labels)

plt.title('Test Accuracy')
plt.xlabel('Running time /s')
plt.ylabel("Accuracy")

plt.xlim([0,axis_lvhba.max()])
plt.savefig(results_path + 'test_accuracy_running_time.pdf', bbox_inches="tight")

In [ ]:
plt.figure(figsize=(8,6))

plt.plot(axis_iter,test_acc_lvhba.mean(0),'-',color = cmap(0),linewidth = 4, label="LV-HBA")
plt.fill_between(axis_iter,test_acc_lvhba.mean(0)-test_acc_lvhba.std(0),test_acc_lvhba.mean(0)+test_acc_lvhba.std(0),color = cmap(0),alpha=0.2)

plt.plot(axis_iter,test_acc_ga.mean(0),'--',color = cmap(2),linewidth = 4, label="GAM")
plt.fill_between(axis_iter,test_acc_ga.mean(0)-test_acc_ga.std(0),test_acc_ga.mean(0)+test_acc_ga.std(0),color = cmap(2),alpha=0.2)


plt.plot(axis_iter,test_acc_blocc.mean(0),'-',color = 'r',linewidth = 4, label="BLOCC")
plt.fill_between(axis_iter,test_acc_blocc.mean(0)-test_acc_blocc.std(0),test_acc_blocc.mean(0)+test_acc_blocc.std(0),color = 'r',alpha=0.2)


plt.legend(ordered_handles, ordered_labels)

plt.title('Test Accuracy')
plt.xlabel('Iteration Count')
plt.ylabel("Accuracy")
plt.savefig(results_path + 'test_accuracy_iteration.pdf', bbox_inches="tight")

In [ ]:
plt.figure(figsize=(8,6))


plt.plot(iter_axis,val_loss_lvhba.mean(0),'-',color = cmap(0), linewidth = 4, label="LV-HBA")
fill_dn_LV = val_loss_lvhba.mean(0)-val_loss_lvhba.std(0)
fill_up_LV = val_loss_lvhba.mean(0)+val_loss_lvhba.std(0)
plt.fill_between(iter_axis[1:],fill_dn_LV[1:],fill_up_LV[1:],color = cmap(0), alpha=0.2)

plt.plot(iter_axis,val_loss_ga.mean(0),'--',color = cmap(2), linewidth = 4, label="GAM")
fill_up_GAM = val_loss_ga.mean(0)-val_loss_ga.std(0)
fill_dn_GAM = val_loss_ga.mean(0)+val_loss_ga.std(0)
plt.fill_between(iter_axis[1:],fill_dn_GAM[1:],fill_up_GAM[1:],color = cmap(2), alpha=0.2)

plt.plot(iter_axis,val_loss_blocc.mean(0),'-',color = 'r', linewidth = 4, label="BLOCC")
fill_dn_blocc = val_loss_blocc.mean(0)-val_loss_blocc.std(0)
fill_up_blocc = val_loss_blocc.mean(0)+val_loss_blocc.std(0)
plt.fill_between(iter_axis[1:],fill_dn_blocc[1:],fill_up_blocc[1:],color = 'r', alpha=0.2)

plt.legend(ordered_handles, ordered_labels)

plt.title('Validation Loss / Upper Level Objective')
plt.xlabel('Iteration Count')
plt.ylabel("Loss")


plt.ylim([1.5,3.4])
plt.savefig(results_path + 'upper_obj_iteration.pdf', bbox_inches="tight")

In [ ]:
plt.figure(figsize=(8,6))


plt.plot(axis_lvhba,val_loss_lvhba.mean(0),'-',color = cmap(0), linewidth = 4, label="LV-HBA")
fill_dn_LV = val_loss_lvhba.mean(0)-val_loss_lvhba.std(0)
fill_up_LV = val_loss_lvhba.mean(0)+val_loss_lvhba.std(0)
plt.fill_between(axis_lvhba[1:],fill_dn_LV[1:],fill_up_LV[1:],color = cmap(0), alpha=0.2)

plt.plot(axis_gam,val_loss_ga.mean(0),'--',color = cmap(2), linewidth = 4, label="GAM")
fill_up_GAM = val_loss_ga.mean(0)-val_loss_ga.std(0)
fill_dn_GAM = val_loss_ga.mean(0)+val_loss_ga.std(0)
plt.fill_between(axis_gam[1:],fill_dn_GAM[1:],fill_up_GAM[1:],color = cmap(2), alpha=0.2)

plt.plot(axis_blocc,val_loss_blocc.mean(0),'-',color = 'r', linewidth = 4, label="BLOCC")
fill_dn_blocc = val_loss_blocc.mean(0)-val_loss_blocc.std(0)
fill_up_blocc = val_loss_blocc.mean(0)+val_loss_blocc.std(0)
plt.fill_between(axis_blocc[1:],fill_dn_blocc[1:],fill_up_blocc[1:],color = 'r', alpha=0.2)

plt.legend(ordered_handles, ordered_labels)
plt.title('Validation Loss (Upper Level)')
plt.xlabel('Running Time /s')
plt.ylabel("Loss")

plt.ylim([1.5,4])
plt.xlim([0,13])
plt.xlim([0,axis_lvhba.max()])
plt.savefig(results_path + 'upper_obj_time.pdf', bbox_inches="tight")

In [ ]:
plt.figure(figsize=(8,6))

fill_dn_blocc = test_loss_blocc.mean(0)-test_loss_blocc.std(0)
fill_up_blocc = test_loss_blocc.mean(0)+test_loss_blocc.std(0)
plt.fill_between(iter_axis[1:],fill_dn_blocc[1:],fill_up_blocc[1:],color = 'r', alpha=0.2)

fill_dn_LV = test_loss_lvhba.mean(0)-test_loss_lvhba.std(0)
fill_up_LV = test_loss_lvhba.mean(0)+test_loss_lvhba.std(0)
plt.fill_between(iter_axis[1:],fill_dn_LV[1:],fill_up_LV[1:],color = cmap(0), alpha=0.2)

fill_up_GAM = test_loss_ga.mean(0)-test_loss_ga.std(0)
fill_dn_GAM = test_loss_ga.mean(0)+test_loss_ga.std(0)
plt.fill_between(iter_axis[1:],fill_dn_GAM[1:],fill_up_GAM[1:],color = cmap(2), alpha=0.2)

plt.plot(iter_axis,test_loss_lvhba.mean(0),'-',color = cmap(0), linewidth = 4, label="LV-HBA")

plt.plot(iter_axis,test_loss_ga.mean(0),'--',color = cmap(2), linewidth = 4, label="GAM")

plt.plot(iter_axis,test_loss_blocc.mean(0),'-',color = 'r', linewidth = 4, label="BLOCC")


plt.legend(ordered_handles, ordered_labels)
plt.title('Test Loss')
plt.xlabel('Iteration Count')
plt.ylabel("Loss")

plt.ylim([1.5,3.4])
plt.savefig(results_path + 'test_obj_iteration.pdf', bbox_inches="tight")

In [ ]:
plt.figure(figsize=(8,6))

plt.plot(iter_axis[1:],lower_loss_blocc.mean(0),'-',color = 'r', linewidth = 4, label="BLOCC")
# fill_dn_blocc = lower_loss_blocc.mean(0)-lower_loss_blocc.std(0)
# fill_up_blocc = lower_loss_blocc.mean(0)+lower_loss_blocc.std(0)
# plt.fill_between(iter_axis[1:],fill_dn_blocc,fill_up_blocc,alpha=0.2)
# the above didn't work as the standard deviation is too small

plt.plot(iter_axis[1:],lower_loss_lvhba.mean(0),'-',color = cmap(0), linewidth = 4, label="LV-HBA")
fill_dn_LV = lower_loss_lvhba.mean(0)-lower_loss_lvhba.std(0)
fill_up_LV = lower_loss_lvhba.mean(0)+lower_loss_lvhba.std(0)
plt.fill_between(iter_axis[1:],fill_dn_LV,fill_up_LV,color = cmap(0),alpha=0.2)

plt.plot(iter_axis[1:],lower_loss_ga.mean(0),'--', color = cmap(2),linewidth = 4, label="GAM")
fill_up_GAM = lower_loss_ga.mean(0)-lower_loss_ga.std(0)
fill_dn_GAM = lower_loss_ga.mean(0)+lower_loss_ga.std(0)
plt.fill_between(iter_axis[1:],fill_dn_GAM,fill_up_GAM,color = cmap(2),alpha=0.2)

plt.legend()
plt.title('Lower Loss')
plt.xlabel('Iteration Count')
plt.ylabel("Loss")

plt.savefig(results_path + 'lower_obj_iteration.pdf', bbox_inches="tight")

In [ ]:
plt.figure(figsize=(8,6))

plt.plot(axis_blocc[1:],lower_loss_blocc.mean(0),'-',color = 'r', linewidth = 4, label="BLOCC")
# fill_dn_blocc = lower_loss_blocc.mean(0)-lower_loss_blocc.std(0)
# fill_up_blocc = lower_loss_blocc.mean(0)+lower_loss_blocc.std(0)
# plt.fill_between(iter_axis[1:],fill_dn_blocc,fill_up_blocc,alpha=0.2)
# the above didn't work as the standard deviation is too small

plt.plot(axis_lvhba[1:],lower_loss_lvhba.mean(0),'-',color = cmap(0), linewidth = 4, label="LV-HBA")
fill_dn_LV = lower_loss_lvhba.mean(0)-lower_loss_lvhba.std(0)
fill_up_LV = lower_loss_lvhba.mean(0)+lower_loss_lvhba.std(0)
plt.fill_between(axis_lvhba[1:],fill_dn_LV,fill_up_LV,color = cmap(0),alpha=0.2)

plt.plot(axis_gam[1:],lower_loss_ga.mean(0),'--', color = cmap(2),linewidth = 4, label="GAM")
fill_up_GAM = lower_loss_ga.mean(0)-lower_loss_ga.std(0)
fill_dn_GAM = lower_loss_ga.mean(0)+lower_loss_ga.std(0)
plt.fill_between(axis_gam[1:],fill_dn_GAM,fill_up_GAM,color = cmap(2),alpha=0.2)

plt.legend()
plt.title('Lower Level Objective')
plt.xlabel('Running Time /s')
plt.ylabel("Loss")
plt.xlim([0,axis_lvhba.max()])

plt.savefig(results_path + 'lower_obj_time.pdf', bbox_inches="tight")